In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu

In [4]:
PATH="./dataset"
train = pd.read_csv(PATH + "/train.csv" , sep='\t',  header=0)
test =  pd.read_csv(PATH + "/test.csv" , sep=',',  header=0)

FileNotFoundError: [Errno 2] No such file or directory: './dataset/train.csv'

In [3]:
display(train.head(3))
print(train.userId.unique().shape[0],train.movieId.unique().shape[0])

,userId,movieId,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960


943 1650


In [4]:
display(test.head(3))
print(test.userId.unique().shape[0],test.movieId.unique().shape[0])

,Id,userId,movieId,rating
0,0,1,6,3
1,1,1,10,3
2,2,1,12,3


459 1410


In [5]:
n_users = np.unique(np.concatenate((train.userId.unique(),test.userId.unique()))).shape[0]
n_items = np.unique(np.concatenate((train.movieId.unique(),test.movieId.unique()))).shape[0]
print(n_items,n_users)

1682 943


In [6]:
train_user_rating_mean = train.groupby(['userId'])['rating'].mean().reset_index(name='mr_user')
train = train.merge(train_user_rating_mean, on=['userId'], how='left', validate='m:1')
train_movie_rating_mean = train.groupby(['movieId'])['rating'].mean().reset_index(name='mr_movie')
train = train.merge(train_movie_rating_mean, on=['movieId'], how='left', validate='m:1')


test = test.merge(train_user_rating_mean, on=['userId'], how='left', validate='m:1')
test = test.merge(train_movie_rating_mean, on=['movieId'], how='left', validate='m:1')

In [7]:
display(train.head(3))

,userId,movieId,rating,timestamp,mr_user,mr_movie
0,1,1,5,874965758,3.681481,3.892950
1,1,2,3,876893171,3.681481,3.180952
2,1,3,4,878542960,3.681481,3.000000


In [8]:
display(test.head(3))
print(test.mr_user.isna().sum(),test.mr_movie.isna().sum())

,Id,userId,movieId,rating,mr_user,mr_movie
0,0,1,6,3,3.681481,3.400000
1,1,1,10,3,3.681481,3.876712
2,2,1,12,3,3.681481,4.398104


0 32


In [9]:
test.mr_movie[test.mr_movie.isna()] = train.mr_movie.median()

/tmp/ipykernel_26123/1761232206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.mr_movie[test.mr_movie.isna()] = train.mr_movie.median()


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(train[['rating','mr_user','mr_movie']])
train[['rating','mr_user','mr_movie']] = scaler.transform(train[['rating','mr_user','mr_movie']])
test[['rating','mr_user','mr_movie']] = scaler.transform(test[['rating','mr_user','mr_movie']])

In [11]:
display(train.head(3))
print(test.mr_user.isna().sum(),test.mr_movie.isna().sum())

,userId,movieId,rating,timestamp,mr_user,mr_movie
0,1,1,1.00,874965758,0.645477,0.723238
1,1,2,0.50,876893171,0.645477,0.545238
2,1,3,0.75,878542960,0.645477,0.500000


0 0


In [13]:
user_rm = Input(shape=[1], name="User-Median-Rating")
movie_rm = Input(shape=[1], name="Movie-Median-Rating")
# creating book embedding path
movie_input = Input(shape=[1], name="Movie-Input")
movie_embedding = Embedding(n_items+1, 5, name="Movie-Embedding")(movie_input)
movie_vec = Flatten(name="Flatten-Books")(movie_embedding)
# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)
# concatenate features
conc = Concatenate()([user_rm, movie_rm, movie_vec, user_vec])
# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)
# Create model and compile it
model2 = Model([user_rm,movie_rm,user_input, movie_input], out)
model2.compile('adam', 'mean_squared_error')

2022-06-20 21:08:19.162982: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-20 21:08:19.163031: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-20 21:08:19.163063: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nico-T420): /proc/driver/nvidia/version does not exist


In [14]:
model2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Movie-Input (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 User-Input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 Movie-Embedding (Embedding)    (None, 1, 5)         8415        ['Movie-Input[0][0]']            
                                                                                                  
 User-Embedding (Embedding)     (None, 1, 5)         4720        ['User-Input[0][0]']             
                                                                                              

In [15]:
history = model2.fit([train.mr_user,train.mr_movie,train.userId, train.movieId], train.rating, epochs=5, verbose=1)

Epoch 1/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0546
Epoch 2/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0518
Epoch 3/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0502
Epoch 4/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0485
Epoch 5/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0467
Epoch 6/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0454
Epoch 7/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0444
Epoch 8/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0436
Epoch 9/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0429
Epoch 10/10
2500/2500 [==============================] - 7s 3ms/step - loss: 0.0423


In [16]:
predictions = model2.predict([test.mr_user,test.mr_movie,test.userId, test.movieId])

In [17]:
predictions

array([[0.6994077 ],
       [0.67993784],
       [0.9052245 ],
       ...,
       [0.6152044 ],
       [0.80706006],
       [0.71773046]], dtype=float32)

In [18]:
test['rating'] = (predictions)
test[['rating','mr_user','mr_movie']] = scaler.inverse_transform(test[['rating','mr_user','mr_movie']])
test['rating'] = np.round(test['rating'])
test.head()

,Id,userId,movieId,rating,mr_user,mr_movie
0,0,1,6,4.0,3.681481,3.400000
1,1,1,10,4.0,3.681481,3.876712
2,2,1,12,5.0,3.681481,4.398104
3,3,1,14,4.0,3.681481,3.900000
4,4,1,17,3.0,3.681481,3.180556


In [19]:
max_rating = test['rating'].max()
min_rating = test['rating'].min()
max_rating, min_rating

(5.0, 1.0)

In [20]:
test

,Id,userId,movieId,rating,mr_user,mr_movie
0,0,1,6,4.0,3.681481,3.400000
1,1,1,10,4.0,3.681481,3.876712
2,2,1,12,5.0,3.681481,4.398104
3,3,1,14,4.0,3.681481,3.900000
4,4,1,17,3.0,3.681481,3.180556
...,...,...,...,...,...,...
19995,19995,458,648,4.0,3.683333,3.962963
19996,19996,458,1101,4.0,3.683333,3.724138
19997,19997,459,934,3.0,3.384615,2.901639
19998,19998,460,10,4.0,3.417910,3.876712


In [21]:
test.drop(['mr_user','mr_movie'],axis=1,inplace=True)
test.to_csv(PATH + "/test_3relu_1.csv", index=False)